# venture backed companies

In [1]:
import requests
import pandas as pd
import tarfile
import datetime
import cnfg

In [4]:
config = cnfg['.crunchbase_config']
user_key = config['user_key']

In [5]:
url = 'https://api.crunchbase.com/v/3/odm/odm.csv.tar.gz?user_key=' + user_key

In [8]:
url

'https://api.crunchbase.com/v/3/odm/odm.csv.tar.gz?user_key=419d2896eaa2ef4a93bff62ae0af51e8'

In [27]:
today = datetime.datetime.today()
filename = 'data/crunchbase_open_map/open_data_map_' + today.strftime('%m%d%y_%H%M%S')
with open(filename, 'wb') as handle:
    response = requests.get(url, stream=True)

    if not response.ok:
        pass

    for block in response.iter_content(1024):
        handle.write(block)
print filename

data/crunchbase_open_map/open_data_map 091515_200034


In [120]:
filename = 'data/crunchbase_open_map/open_data_map 091515_200034'

In [121]:
tfile = tarfile.open(filename, 'r:gz')
orgs = tfile.extractfile('organizations.csv')


In [122]:
df = pd.read_csv(orgs, encoding='utf8')
df.head()

,crunchbase_uuid,type,primary_role,name,crunchbase_url,homepage_domain,homepage_url,profile_image_url,facebook_url,twitter_url,linkedin_url,stock_symbol,location_city,location_region,location_country_code,short_description
0,e139350830ea8a363f96dd3226033abd,Organization,company,Wetpaint,https://www.crunchbase.com/organization/wetpai...,wetpaint-inc.com,http://wetpaint-inc.com,https://www.crunchbase.com/organization/wetpai...,NaN,http://twitter.com/BachelrWetpaint,NaN,NaN,Seattle,Washington,USA,Wetpaint offers an online social publishing pl...
1,bf4d7b0eb34d2fd8d2926049c4f7efc7,Organization,company,Zoho,https://www.crunchbase.com/organization/zoho?u...,zoho.com,http://zoho.com,https://www.crunchbase.com/organization/zoho/p...,http://www.facebook.com/zoho,http://twitter.com/zoho,http://www.linkedin.com/company/zoho-corporati...,NaN,Pleasanton,California,USA,"Zoho offers a suite of business, collaboration..."
2,5f2b40b8d1b3d323d81ab7a8e89553d0,Organization,company,Digg,https://www.crunchbase.com/organization/digg?u...,digg.com,http://www.digg.com,https://www.crunchbase.com/organization/digg/p...,http://www.facebook.com/digg,http://twitter.com/digg,http://www.linkedin.com/company/digg,NaN,New York,New York,USA,Digg Inc. operates a website that enables its ...
3,f4d5ab44058b298bea81380e6e9a8eec,Organization,investor,Omidyar Network,https://www.crunchbase.com/organization/omidya...,omidyar.com,http://www.omidyar.com,https://www.crunchbase.com/organization/omidya...,http://www.facebook.com/OmidyarNetwork,http://twitter.com/OmidyarNetwork,http://www.linkedin.com/company/22806,NaN,Redwood City,California,USA,"Pierre Omidyar, the founder of eBay, and his w..."
4,df6628127f970b439d3e12f64f504fbb,Organization,company,Facebook,https://www.crunchbase.com/organization/facebo...,facebook.com,http://www.facebook.com,https://www.crunchbase.com/organization/facebo...,https://www.facebook.com/,https://twitter.com/facebook,http://www.linkedin.com/company/facebook,NaN,NaN,NaN,NaN,Facebook is an online social networking servic...


In [123]:
tfile.close()

In [124]:
private = df[df.primary_role == 'company'][['name', 'location_city', 'location_region', 'location_country_code']]
private.to_pickle('companies_venture_df.pkl')

In [132]:
private.head()

,name,location_city,location_region,location_country_code
0,Wetpaint,Seattle,Washington,USA
1,Zoho,Pleasanton,California,USA
2,Digg,New York,New York,USA
4,Facebook,NaN,NaN,NaN
8,Omnidrive,Palo Alto,California,USA


In [125]:
private = pd.read_pickle('companies_venture_df.pkl')

# full text searching instead of NER

In [60]:
import textual
import companies
import string
from unidecode import unidecode
import nltk
reload(textual)

<module 'textual' from 'textual.pyc'>

In [128]:
entities = pd.read_pickle('entities_df')
ents = entities['entities']
texts = entities['raw_text']

In [133]:
print texts.ix[0]


SUMMARY: 
Notice is hereby given of the following determinations: Pursuant to the authority vested in me by the Act of October 19, 1965 (79 Stat. 985; 22 U.S.C. 2459), Executive Order 12047 of March 27, 1978, the Foreign Affairs Reform and Restructuring Act of 1998 (112 Stat. 2681, et seq.; 22 U.S.C. 6501 note, et seq. ), Delegation of Authority No. 234 of October 1, 1999, Delegation of Authority No. 236-3 of August 28, 2000 (and, as appropriate, Delegation of Authority No. 257 of April 15, 2003), I hereby determine that the objects to be included in the exhibition “Girl with a Pearl Earring: Dutch Paintings from the Mauritshuis,” imported from abroad for temporary exhibition within the United States, are of cultural significance. The objects are imported pursuant to loan agreements with the foreign owner or custodian. I also determine that the exhibition or display of the exhibit objects at the Fine Arts Museums of San Francisco, San Francisco, CA, from on or about January 26, 2013, 

In [142]:
text = texts.ix[0]
tokens = nltk.word_tokenize(texts.ix[0])
for i in private.name.dropna():
    if len(i.split()) > 1:
        # search original text if company name is more than one word
        if text.find(i) != -1: print i
    else:
        # search tokens if company name is a single word
        if i in tokens: print i

York
23
U.S. Department of State
Office
State
August
Stat
to be
August
Reform
Collection
Office
.
.
Pearl
High Museum of Art
June
Pearl


In [143]:
def smart_find(haystack, needle):
    if haystack.startswith(needle+" "):
        return True
    if haystack.endswith(" "+needle):
        return True
    if haystack.find(" "+needle+" ") != -1:
        return True
    return False

text = texts.ix[0]
text = unidecode(text).translate(None, string.punctuation)

# U.S. state department is missed because of punctuation removal
# punctuation must be removed to account for words that end in a period or a comma in the text

for i in private.name.dropna():
    if smart_find(text, i):
        print i

York
23
Office
State
August
Stat
to be
August
Reform
Collection
Office
Pearl
High Museum of Art
June
Pearl


In [144]:
terms = {i[0] for i in ents[0] if i[1] in ['ORGANIZATION', 'PERSON']}
matched = companies.find_companies(terms)
private[private.name.isin(matched)]

us department of state
high museum of art


,name,location_city,location_region,location_country_code
62770,U.S. Department of State,Washington,District of Columbia,USA
271488,High Museum of Art,Atlanta,Georgia,USA


#public companies

In [96]:
names = pd.read_csv('data/corpwatch_dump/companies.csv', sep='\t')
names = names[['cw_id', 'cik', 'company_name']]

In [102]:
locations = pd.read_csv('data/corpwatch_dump/company_locations.csv', sep='\t')
# prioritizes business and mailing address information of state of incorporation info
locations = locations.sort(['type', 'date'], ascending=[True, False]).drop_duplicates('cw_id')
locations = locations[['cw_id', u'city', 'country_code', u'subdiv_code']]

In [103]:
names_loc = pd.merge(names, locations, how='left')

In [109]:
info = pd.read_csv('data/corpwatch_dump/company_info.csv', sep='\t')
info = info.sort('year', ascending=False).drop_duplicates('cw_id')
info = info[['cw_id', 'industry_name', 'sector_name']]

In [110]:
names_loc_info = pd.merge(names_loc, info, how='left')

In [116]:
tickers = pd.read_csv('data/rank_and_filed_tickers_dump/cik_ticker.csv', sep='|')
tickers = tickers[['CIK', 'Ticker']]
tickers.columns = ['cik', 'ticker']

In [117]:
public_companies = pd.merge(names_loc_info, tickers, how='left')

In [131]:
public_companies.head()

,cw_id,cik,company_name,city,country_code,subdiv_code,industry_name,sector_name,ticker
0,781188,13,CORPORATE INCOME FUND SEVENTY NINTH SHORT TERM...,NEW YORK,US,NY,NaN,NaN,NaN
1,1,20,K TRON INTERNATIONAL INC,PITMAN,US,NJ,"Industrial instruments for measurement, displa...",Instruments and related products,KTII
2,2,1750,AAR CORP,WOOD DALE,US,IL,Aircraft & parts,Transportation equipment,AIR
3,3,1800,ABBOTT LABORATORIES,ABBOTT PARK,US,IL,Pharmaceutical preparations,Chemicals and allied products,ABT
4,4,1841,ABEL NOSER CORP,NEW YORK,US,NY,NaN,NaN,NaN


In [119]:
public_companies.to_pickle('companies_public_df.pkl')